In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re
from pathlib import Path

In [2]:
import sys
sys.path.append('..\\..\\')

In [3]:
import sys
path_to_pdn = os.path.join(Path.home(), 'Documents\GitHub\pdn___')
# сюда внутри кавычек вставляем путь, где ледит репозиторий пдн
sys.path.insert(0, path_to_pdn)

In [4]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta
from pdn.projects.emias_data import (
    get_emias_data,
    get_emias_data_july,
    get_emias_id_and_task_num_and_uschedule_visit,
    get_status_rec_rcp_procedure_and_doc,
    get_uschedule_visit_attr
)
from pdn.projects.patients_data import get_status_patients
from pdn.downloading_files import downloads_from_nextcloud
from pdn.projects.calls_data import get_preprocess_calls_data, get_last_comment_calls
from pdn.configs.paths import CALLS_SUBFOLDER_LOADS, TRASH_FOLDER, PATH_PROJECT
from pdn.configs.columns import out_cols
from pdn.reading.read_files import get_phone_specific_pats, get_group_name_df
from pdn.data_preprocess.preprocess_emias_data import preprocess_phone_numbers
from pdn.data_preprocess.data_attrs import add_cols_task_attrs, get_df_with_hospital_smp_attr
from pdn.utils.writing import write_df_for_call_programm, save_file_if_not_exist
from pdn.reading.read_files import get_stop_pdn_pats
from pdn.utils.utils import get_filenames
from pdn.reading.read_files import read_xlsx_files
%load_ext autoreload
%autoreload 2

In [5]:
out_task_cur_all = pd.DataFrame()

In [6]:
cycle = 1

In [7]:
cycle = 1
date_now = pd.Timestamp.today()
# - timedelta(days=1)
if cycle in [1, 2]:
        task_for_call = list(range(1, 34))
else:
    task_for_call = list(range(1, 13)) + list(range(14, 20)) + [22, 23, 24, 25]
ther_exclude_dates_june = [str(val)[:10] for val in pd.date_range('2022-01-01', '2022-06-07')]
ther_exclude_dates_july = [str(val)[:10] for val in pd.date_range('2022-01-01', '2022-07-04')]
date_ymd = date_now.strftime('%Y%m%d')
yesterday_date_ = date_now - timedelta(days=1)

In [8]:
month_prev = 'august'
month_prev_for_emias = 3833
month_cur = 'september'
month_cur_for_emias = 4059
month_income = 'income'
month_income_for_emias = 1111

In [9]:
part = 1

# Пациенты предыдущего месяца

patients_emias_prev = get_emias_data(date_now, drop_exclude_pats=False, month=month_prev_for_emias)
patients_db_prev = get_status_patients(month_prev)
patients_prev = patients_emias_prev.merge(
    patients_db_prev[['emiasid', 'status', 'old_comment', 'next_call_date', 'operator_login']], 
    on='emiasid', 
    how='right'
)
for col in [
    'kont_priyom_terapevt_zapis',
    'kont_priyom_terapevt_sdel',
    'kont_priyom_terapevt_sdel_hron_plan'
]:
    patients_emias_prev.loc[patients_emias_prev[col].isin(ther_exclude_dates_june), [col]] = np.nan
attrs_emias_data = patients_prev.merge(
    get_emias_id_and_task_num_and_uschedule_visit(
        patients_prev,
        date_now
    ),
    how='left'
).merge(
    get_status_rec_rcp_procedure_and_doc(# таблица с признаками 
        get_emias_data(date_now, drop_exclude_pats=True, preprocess_april=False, month=month_prev_for_emias),
        date_now
    ),
    how='left'
)
attrs_emias_data.next_call_date = pd.to_datetime(attrs_emias_data.next_call_date, dayfirst=True).dt.strftime('%Y-%m-%d')

out = attrs_emias_data.merge(
    get_group_name_df(),
    how='left',
    on=['group'])

preprocess_phone_numbers(out)

out.birth_date = pd.to_datetime(out.birth_date, dayfirst=True).dt.strftime('%Y-%m-%d')

out.loc[out.next_call_date == date_now.strftime('%Y-%m-%d'), ['task_to_call']] = 'Да'
out_prev = out.copy()

out.shape

# Файлы на загрузку

out_task_prev = out_prev[out_cols]
out_task_prev.shape[0]

# основной файл для загрузки в звонилку
write_df_for_call_programm(
    out_task_prev,
    os.path.join(PATH_PROJECT, 'data', 'task', rf'{month_prev}_{date_now.strftime("%Y%m%d")}_v1.xlsx'))

# Пациенты текущего месяца

In [10]:
patients_emias_cur = get_emias_data(date_now, drop_exclude_pats=False, month=month_cur_for_emias)
patients_emias_cur.shape[0]

97159

In [11]:
patients_emias_cur.columns

Index(['Unnamed: 0', 'MO_ID', 'mo', 'FIL_ID', 'mu', 'MU_id', 'mo_sozd_prog',
       'emiasid', 'family', 'name',
       ...
       'Умер', 'Признак наличия госпитализации', 'Дата госпитализации',
       'Краткое наименование стационарной медицинской организации',
       'Краткое наименование отделения', 'Диагноз при поступлении',
       'dia_group_names', 'reason_exclude', 'AGE', 'AGE_GROUP'],
      dtype='object', length=121)

# ПРИ ДОБАВЛЕНИИ ЗАДАНИЯ ПЕРЕЗАПУСКАТЬ ОТСЮДА ИЗМЕНИВ ВЕРСИЮ

In [12]:
patients_db_cur = get_status_patients(month_cur)
patients_db_cur.shape[0]

767

In [13]:
patients_db_cur['next_call_date'] = date_now.strftime('%Y-%m-%d')

In [14]:
%%time
patients_cur = patients_emias_cur.merge(
    patients_db_cur[['emiasid', 'status', 'old_comment', 'operator_login', 'next_call_date']], 
    on='emiasid', 
    how='right'
)
for col in [
    'kont_priyom_terapevt_zapis',
    'kont_priyom_terapevt_sdel',
    'kont_priyom_terapevt_sdel_hron_plan'
]:
    patients_emias_cur.loc[patients_emias_cur[col].isin(ther_exclude_dates_july), [col]] = np.nan
attrs_emias_data = patients_cur.merge(
    get_emias_id_and_task_num_and_uschedule_visit(
        patients_cur,
        date_now
    ),
    how='left'
).merge(
    get_status_rec_rcp_procedure_and_doc(# таблица с признаками 
        get_emias_data(date_now, drop_exclude_pats=True, preprocess_april=False, month=month_cur_for_emias),
        date_now
    ),
    how='left'
)
attrs_emias_data.next_call_date = pd.to_datetime(attrs_emias_data.next_call_date, dayfirst=True).dt.strftime('%Y-%m-%d')

out = attrs_emias_data.merge(
    get_group_name_df(),
    how='left',
    on=['group'])

preprocess_phone_numbers(out)

out.birth_date = pd.to_datetime(out.birth_date, dayfirst=True).dt.strftime('%Y-%m-%d')

C:\Users\VAIO\Documents\GitHub\pdn___\pdn\data_preprocess\preprocess_emias_data.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  melt_data['variable'] = melt_data['variable'].str.replace('_sdel|_zapis', '')
C:\Users\VAIO\Documents\GitHub\pdn___\pdn\data_preprocess\preprocess_emias_data.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  melt_data['variable'] = melt_data['variable'].str.replace('_sdel|_zapis', '')
C:\Users\VAIO\Documents\GitHub\pdn___\pdn\data_preprocess\preprocess_emias_data.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  melt_data['variable'] = melt_data['variable'].str.replace('_sdel|_zapis', '')
C:\Users\VAIO\Documents\GitHub\pdn___\pdn\projects\emias_data.py:198: FutureWarning: The default value of regex will change from True to False in a future version.
  data_attr_by_pat[col] = data_attr_by_pat[col].astyp

CPU times: total: 1min 54s
Wall time: 1min 54s


In [15]:
out.loc[out.status == 'Напомнить', ['patient_group']] = 'Напомнить о записи на исследование на завтра'

In [16]:
# out.loc[out.status == 'Внеплановый', ['patient_group']] = 'Внеплановый'

In [17]:
out.loc[out.next_call_date == date_now.strftime('%Y-%m-%d'), ['task_to_call']] = 'Да'

In [18]:
out_cur = out.copy()
out_cur.shape

(767, 134)

# Файлы на загрузку

In [19]:
out_task_cur = out_cur[out_cols]
out_task_cur.shape[0]

767

In [20]:
out_task_cur = out_task_cur.drop_duplicates(subset='emiasid', keep='last')

In [21]:
out_task_cur.shape[0]

763

In [22]:
part

1

In [23]:
out_task_cur.operator_login = out_task_cur.operator_login.apply(lambda x: x + '_n2')

In [ ]:
%%time
# основной файл для загрузки в звонилку
write_df_for_call_programm(
    out_task_cur,
    os.path.join(PATH_PROJECT, 'data', 'task', rf'{month_cur}_{date_now.strftime("%Y%m%d")}_part_{part}.xlsx'))
part += 1

In [ ]:
out_task_cur.emiasid.nunique()